In [ ]:
# Скрипт для выполнения предсказания на валидационных данных
# с замером времени.

# ==============================================================================
# 0. Импорт необходимых библиотек
# ==============================================================================

import pandas as pd
import numpy as np
import pickle
import time # Для замера времени
import sys
import gc
from pathlib import Path

# Импорты sklearn (необходимы для функций предобработки и загрузки модели)
from sklearn.model_selection import train_test_split # Хотя train_test_split не используется в предсказании, он часть preprocess_data
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score # Не используется для предсказания, но может быть в загруженной модели

# Импорт LGBMClassifier (необходим для загрузки модели)
from lightgbm import LGBMClassifier

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)


# ==============================================================================
# 1. Определение вспомогательных функций (Скопированы из полного пайплайна)
#    Примечание: Эти функции нужны для работы preprocess_data.
# ==============================================================================

def process_notset_none(df: pd.DataFrame):
    """
    Обнаруживает строки 'not set' и 'none' и заменяет на np.nan. In-place.
    """
    if not isinstance(df, pd.DataFrame): return # Добавлена проверка
    strings_to_find_and_replace = ['not set', 'none']
    # print("Начало обработки 'not set' и 'none'", file=sys.stderr)
    for col in df.columns:
        col_series_str = df[col].astype(str)
        combined_mask = col_series_str.str.contains(strings_to_find_and_replace[0], case=False, regex=False) | \
                        col_series_str.str.contains(strings_to_find_and_replace[1], case=False, regex=False)
        if combined_mask.sum() > 0:
            df.loc[combined_mask, col] = np.nan
    # print("Обработка 'not set' и 'none' завершена.", file=sys.stderr)

def encode_column_with_nulls(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """Кодирует столбец числовыми метками (NaN->0, остальное 1+). Возвращает новый DF."""
    df = df.copy()
    if column_name not in df.columns: return df
    encoded, uniques = pd.factorize(df[column_name].astype(str))
    df[f'{column_name}_encoded'] = np.where(encoded == -1, 0, encoded + 1)
    df[f'{column_name}_encoded'] = df[f'{column_name}_encoded'].astype('int64')
    return df

def get_mode(x):
    """Вспомогательная функция для моды."""
    mode_val = x.mode()
    if not mode_val.empty: return mode_val[0]
    return np.nan

def fill_missing_by_groups(df: pd.DataFrame) -> pd.DataFrame:
    """Заполняет пропуски по группам и модой. In-place."""
    # print("Начало заполнения пропусков по группам...", file=sys.stderr)
    if 'utm_source' in df.columns and 'utm_medium' in df.columns and 'device_category' in df.columns and df['utm_source'].isna().any():
        group_modes_source = df.groupby(['utm_medium', 'device_category'])['utm_source'].transform(get_mode)
        df['utm_source'].fillna(group_modes_source, inplace=True)
    if 'utm_campaign' in df.columns and 'utm_medium' in df.columns and 'device_os' in df.columns and df['utm_campaign'].isna().any():
         group_modes_campaign = df.groupby(['utm_medium', 'device_os'])['utm_campaign'].transform(get_mode)
         df['utm_campaign'].fillna(group_modes_campaign, inplace=True)

    group_columns_broad = ['utm_medium', 'device_category', 'device_os']
    if all(col in df.columns for col in group_columns_broad):
        cols_with_nan_after_specific = df.columns[df.isna().any()].tolist()
        for col in cols_with_nan_after_specific:
             if col in ['utm_source', 'utm_campaign'] and not df[col].isna().any(): continue
             group_modes_broad_col = df.groupby(group_columns_broad)[col].transform(get_mode)
             df[col].fillna(group_modes_broad_col, inplace=True)

    cols_with_nan_final = df.columns[df.isna().any()].tolist()
    for col in cols_with_nan_final:
        mode_val = df[col].mode()
        if not mode_val.empty: df[col].fillna(mode_val[0], inplace=True)
    # print("Заполнение пропусков завершено.", file=sys.stderr)
    return df

def fill_hits_and_events(df: pd.DataFrame) -> pd.DataFrame:
    """Заполняет пропуски в hit_referer и event_label. In-place."""
    # print("Начало заполнения пропусков в hits...", file=sys.stderr)
    if 'hit_referer' in df.columns and 'hit_page_path' in df.columns and df['hit_referer'].isna().any():
        referer_by_page = df.groupby('hit_page_path')['hit_referer'].transform('first')
        df['hit_referer'].fillna(referer_by_page, inplace=True)
    if 'event_label' in df.columns and 'event_action' in df.columns and df['event_label'].isna().any():
        label_by_action = df.groupby('event_action')['event_label'].transform('first')
        df['event_label'].fillna(label_by_action, inplace=True)
    if 'hit_referer' in df.columns and df['hit_referer'].isna().any():
        df['hit_referer'].fillna('direct', inplace=True)
    if 'event_label' in df.columns and df['event_label'].isna().any():
        df['event_label'].fillna('none', inplace=True)
    # print("Заполнение пропусков в hits завершено.", file=sys.stderr)
    return df

def aggregate_session_data(df: pd.DataFrame) -> pd.DataFrame:
    """Агрегирует данные хитов до уровня сессий."""
    # print("Начало агрегации данных хитов...", file=sys.stderr)
    if 'session_id' not in df.columns:
         print("Ошибка: 'session_id' отсутствует для агрегации.", file=sys.stderr)
         return pd.DataFrame()

    agg_rules = {
        'hit_number': ['min', 'max', 'count'],
        **({'hit_page_path': lambda x: x.iloc[0] if not x.empty else np.nan} if 'hit_page_path' in df.columns else {}),
        **({'event_category_grouped': lambda x: x.value_counts().index[0] if not x.empty else np.nan} if 'event_category_grouped' in df.columns else {}),
        **({'event_action_grouped': lambda x: x.value_counts().index[0] if not x.empty else np.nan} if 'event_action_grouped' in df.columns else {}),
        **({'hit_time_2': 'sum'} if 'hit_time_2' in df.columns else {}),
        **({'hit_referer_encoded': lambda x: x.value_counts().index[0] if not x.empty else np.nan} if 'hit_referer_encoded' in df.columns else {}),
        **({'event_label_encoded': lambda x: x.value_counts().index[0] if not x.empty else np.nan} if 'event_label_encoded' in df.columns else {}),
    }
    utm_columns_in_hits = [col for col in df.columns if col.startswith('utm_') and col not in agg_rules]
    for col in utm_columns_in_hits:
         agg_rules[col] = lambda x: x.iloc[0] if not x.empty else np.nan

    agg_rules = {key: value for key, value in agg_rules.items() if key in df.columns} # Финальная проверка наличия колонок


    try:
        aggregated = df.groupby('session_id').agg(agg_rules)
    except Exception as e:
        print(f"Ошибка при агрегации: {e}", file=sys.stderr)
        return pd.DataFrame()

    # === Логика переименования ===
    rename_map = {}
    for original_col_tuple in aggregated.columns.values:
        if isinstance(original_col_tuple, tuple):
            original_name = original_col_tuple[0]
            agg_name = original_col_tuple[1]
            col_key = original_col_tuple
            final_name = str(original_col_tuple) # По умолчанию

            if original_name == 'hit_number':
                if agg_name == 'min': final_name = 'first_hit_number'
                elif agg_name == 'max': final_name = 'last_hit_number'
                elif agg_name == 'count': final_name = 'total_hits'
            elif original_name == 'hit_time_2' and agg_name == 'sum': final_name = 'total_time'
            elif agg_name == '<lambda>':
                 if original_name == 'hit_page_path': final_name = 'entry_page'
                 elif original_name == 'event_category_grouped': final_name = 'main_category_grouped'
                 elif original_name == 'event_action_grouped': final_name = 'main_action_grouped'
                 elif original_name == 'hit_referer_encoded': final_name = 'main_referer'
                 elif original_name == 'event_label_encoded': final_name = 'main_label'
                 elif original_name in utm_columns_in_hits: final_name = original_name
                 else: final_name = f"{original_name}_lambda_agg"
            else: final_name = f"{original_name}_{agg_name}"
            rename_map[col_key] = final_name
        else:
            rename_map[original_col_tuple] = str(original_col_tuple) # Для SingleIndex


    try:
        aggregated.columns = [rename_map.get(col, str(col)) for col in aggregated.columns]
    except Exception as e:
        print(f"Ошибка при применении переименования столбцов: {e}", file=sys.stderr)
        aggregated.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else str(col) for col in aggregated.columns.values]


    aggregated = aggregated.reset_index()

    # print("Агрегация завершена.", file=sys.stderr)
    return aggregated

def optimize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """Оптимизирует типы данных DataFrame. In-place."""
    # print("  Оптимизация типов данных...", file=sys.stderr)
    # initial_memory = df.memory_usage(deep=True).sum() / (1024 * 1024)
    for col in df.columns:
        if df[col].dtype == 'object':
            if df[col].nunique() / len(df) < 0.1:
                 try: df[col] = df[col].astype('category')
                 except Exception as e: print(f"  Не удалось преобразовать '{col}' в категорию: {e}", file=sys.stderr)
        elif pd.api.types.is_integer_dtype(df[col]):
             try: df[col] = pd.to_numeric(df[col], downcast='integer')
             except Exception as e: print(f"  Не удалось downcast '{col}' (integer): {e}", file=sys.stderr)
        elif pd.api.types.is_float_dtype(df[col]):
             try: df[col] = pd.to_numeric(df[col], downcast='float')
             except Exception as e: print(f"  Не удалось downcast '{col}' (float): {e}", file=sys.stderr)
    # final_memory = df.memory_usage(deep=True).sum() / (1024 * 1024)
    # print(f"  Память после оптимизации: {final_memory:.2f} MB", file=sys.stderr)
    return df

def merge_and_save(sessions_df: pd.DataFrame, hits_df: pd.DataFrame, output_pkl: str = None) -> pd.DataFrame:
    """Объединяет сессии и хиты, опционально сохраняет. Возвращает DF."""
    # print("Начало объединения данных...", file=sys.stderr)
    sessions_df = optimize_dtypes(sessions_df)
    hits_df = optimize_dtypes(hits_df)

    if 'session_id' not in sessions_df.columns or 'session_id' not in hits_df.columns:
        print("Ошибка: Столбец 'session_id' отсутствует в одном из датафреймов для объединения.", file=sys.stderr)
        return pd.DataFrame()

    try:
        merged_df = pd.merge(sessions_df, hits_df, on='session_id', how='left')
        # print("Объединение завершено.", file=sys.stderr)
    except Exception as e:
        print(f"Ошибка при объединении данных: {e}", file=sys.stderr)
        return pd.DataFrame()

    merged_df = optimize_dtypes(merged_df)

    if output_pkl:
        # print(f"Сохранение данных в {output_pkl}...", file=sys.stderr)
        try:
            with open(output_pkl, 'wb') as f:
                pickle.dump(merged_df, f, protocol=pickle.HIGHEST_PROTOCOL)
            # pkl_size = Path(output_pkl).stat().st_size / (1024 * 1024)
            # print(f"Файл сохранен. Размер: {pkl_size:.2f} MB", file=sys.stderr)
        except Exception as e:
            print(f"Ошибка при сохранении файла {output_pkl}: {e}", file=sys.stderr)

    # print("Объединение и сохранение завершены.", file=sys.stderr)
    return merged_df


def reduce_cardinality(df: pd.DataFrame, categorical_features: list, max_categories: int = 50) -> pd.DataFrame:
    """Уменьшает кардинальность категориальных признаков."""
    df = df.copy()
    for col in categorical_features:
        if col in df.columns and df[col].dtype == 'object':
             if df[col].nunique() > max_categories:
                top_categories = df[col].value_counts().nlargest(max_categories-1).index
                if 'OTHER' in top_categories:
                     top_categories = df[col].value_counts().nlargest(max_categories).index
                     if 'OTHER' in top_categories: top_categories = top_categories.drop('OTHER')[:max_categories-1]
                df[col] = np.where(df[col].isin(top_categories), df[col], 'OTHER')
                try: df[col] = df[col].astype('category')
                except Exception as e: print(f"Не удалось преобразовать '{col}' в категорию после reduce_cardinality: {e}", file=sys.stderr)
             elif df[col].nunique() > 0:
                 try: df[col] = df[col].astype('category')
                 except Exception as e: print(f"Не удалось преобразовать '{col}' в категорию: {e}", file=sys.stderr)
    return df

def filter_rare_classes(y: pd.Series, min_samples: int = 2) -> pd.Index:
    """Возвращает индекс классов с достаточным количеством образцов."""
    if not isinstance(y, pd.Series) or y.empty: return pd.Index([])
    value_counts = y.value_counts()
    valid_classes = value_counts[value_counts >= min_samples].index
    if pd.isna(valid_classes).any(): valid_classes = valid_classes.dropna()
    return valid_classes


# ==============================================================================
# 2. Определение признаков для модели (Скопированы из полного пайплайна)
#    Примечание: Эти списки должны точно соответствовать тем, что использовались при обучении.
# ==============================================================================

numeric_features = [
    'visit_number', 'day_of_week_num', 'month', 'day',
    'utm_source_encoded', 'utm_campaign_encoded',
    'first_hit_number', 'last_hit_number', 'total_hits', 'total_time',
    'main_referer', 'main_label', 'geo_country'
]

categorical_features = [
    'utm_medium', 'device_category', 'device_brand',
    'geo_city_grouped', 'entry_page', 'main_category_grouped',
]

# Имя целевого столбца (не признак)
target_column_name_for_prediction = 'main_action_grouped'


# ==============================================================================
# 3. Функция предобработки данных (Скопирована из полного пайплайна)
#    Инкапсулирует весь пайплайн предобработки данных.
# ==============================================================================

def preprocess_data(ga_sessions_df_raw: pd.DataFrame, ga_hits_df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Выполняет полную предобработку данных сессий и хитов, объединяет их
    и возвращает датафрейм, готовый для обучения или предсказания.
    Не включает начальную загрузку файлов и process_notset_none,
    а также финальное обучение/предсказание.
    """
    # print("\n--- Запуск функции preprocess_data ---", file=sys.stderr)

    # Создаем копии для работы внутри функции
    ga_sessions_filled = ga_sessions_df_raw.copy()
    ga_hits_filled = ga_hits_df_raw.copy()

    # --------------------------------------------------------------------------
    # Предобработка данных сессий (ga_sessions_filled)
    # --------------------------------------------------------------------------
    # print("-> Предобработка данных сессий...", file=sys.stderr)
    if 'visit_date' in ga_sessions_filled.columns:
        try:
            ga_sessions_filled['visit_date'] = pd.to_datetime(ga_sessions_filled['visit_date'], errors='coerce') # errors='coerce' заменит некорректные даты на NaT
            ga_sessions_filled.dropna(subset=['visit_date'], inplace=True) # Удалим строки с некорректными датами
            if not ga_sessions_filled.empty:
                ga_sessions_filled['year'] = ga_sessions_filled['visit_date'].dt.year.astype('int16')
                ga_sessions_filled['month'] = ga_sessions_filled['visit_date'].dt.month.astype('int16')
                ga_sessions_filled['day'] = ga_sessions_filled['visit_date'].dt.day.astype('int16')
                ga_sessions_filled['day_of_week_num'] = ga_sessions_filled['visit_date'].dt.dayofweek.astype('int16')
        except Exception as e: print(f"Ошибка при создании признаков даты: {e}", file=sys.stderr)

    ga_sessions_filled = fill_missing_by_groups(ga_sessions_filled)

    if 'utm_source' in ga_sessions_filled.columns: ga_sessions_filled = encode_column_with_nulls(ga_sessions_filled, 'utm_source')
    if 'utm_campaign' in ga_sessions_filled.columns: ga_sessions_filled = encode_column_with_nulls(ga_sessions_filled, 'utm_campaign')
    if 'device_model' in ga_sessions_filled.columns: ga_sessions_filled = encode_column_with_nulls(ga_sessions_filled, 'device_model')

    if 'geo_country' in ga_sessions_filled.columns:
        ga_sessions_filled['geo_country'] = ga_sessions_filled['geo_country'].astype(str).apply(lambda x: 1 if x == 'Russia' else 0)

    if 'geo_city' in ga_sessions_filled.columns and not ga_sessions_filled.empty:
        threshold = len(ga_sessions_filled) * 0.01
        value_counts = ga_sessions_filled['geo_city'].value_counts()
        replace_dict = {category: 'OTHER' for category in value_counts[value_counts < threshold].index}
        ga_sessions_filled['geo_city_grouped'] = ga_sessions_filled['geo_city'].replace(replace_dict)
    elif 'geo_city' in ga_sessions_filled.columns: # Handle empty df but column exists
         ga_sessions_filled['geo_city_grouped'] = 'OTHER'

    # print("-> Предобработка данных сессий завершена.", file=sys.stderr)


    # --------------------------------------------------------------------------
    # Предобработка и агрегация данных хитов (ga_hits_aggregated)
    # --------------------------------------------------------------------------
    # print("\n-> Предобработка и агрегация данных хитов...", file=sys.stderr)
    ga_hits_filled = fill_hits_and_events(ga_hits_filled)

    if 'hit_time' in ga_hits_filled.columns:
        ga_hits_filled['hit_time_2'] = ga_hits_filled['hit_time'].notna().astype(int)

    if 'hit_referer' in ga_hits_filled.columns: ga_hits_filled = encode_column_with_nulls(ga_hits_filled, 'hit_referer')

    if 'event_category' in ga_hits_filled.columns and not ga_hits_filled.empty:
        threshold = len(ga_hits_filled) * 0.01
        value_counts = ga_hits_filled['event_category'].value_counts()
        replace_dict = {category: 'OTHER' for category in value_counts[value_counts < threshold].index}
        ga_hits_filled['event_category_grouped'] = ga_hits_filled['event_category'].replace(replace_dict)
    elif 'event_category' in ga_hits_filled.columns: # Handle empty df but column exists
        ga_hits_filled['event_category_grouped'] = 'OTHER'

    if 'event_action' in ga_hits_filled.columns and not ga_hits_filled.empty:
        threshold = len(ga_hits_filled) * 0.01
        value_counts = ga_hits_filled['event_action'].value_counts()
        replace_dict = {category: 'OTHER' for category in value_counts[value_counts < threshold].index}
        ga_hits_filled['event_action_grouped'] = ga_hits_filled['event_action'].replace(replace_dict)
    elif 'event_action' in ga_hits_filled.columns: # Handle empty df but column exists
         ga_hits_filled['event_action_grouped'] = 'OTHER'

    if 'event_label' in ga_hits_filled.columns: ga_hits_filled = encode_column_with_nulls(ga_hits_filled, 'event_label')

    columns_to_drop_from_hits_filled = ['event_value','hit_date','hit_type', 'hit_time','hit_referer','event_label']
    existing_cols_to_drop_hits = [col for col in columns_to_drop_from_hits_filled if col in ga_hits_filled.columns]
    if existing_cols_to_drop_hits:
        ga_hits_filled = ga_hits_filled.drop(columns=existing_cols_to_drop_hits, axis = 1)

    columns_to_drop_from_sessions_filled = ['visit_date', 'visit_time', 'utm_source','utm_campaign','utm_adcontent','utm_keyword','device_screen_resolution','device_os','geo_city', 'year']
    existing_cols_to_drop_sessions = [col for col in columns_to_drop_from_sessions_filled if col in ga_sessions_filled.columns]
    if existing_cols_to_drop_sessions:
        ga_sessions_filled = ga_sessions_filled.drop(columns=existing_cols_to_drop_sessions, axis = 1)


    ga_hits_aggregated = aggregate_session_data(ga_hits_filled)
    del ga_hits_filled; gc.collect()
    if ga_hits_aggregated.empty:
         print("Предупреждение: ga_hits_aggregated пуст после агрегации.", file=sys.stderr)


    columns_to_drop_from_hits_aggregated = ['main_category','main_action']
    existing_cols_to_drop_hits_agg = [col for col in columns_to_drop_from_hits_aggregated if col in ga_hits_aggregated.columns]
    if existing_cols_to_drop_hits_agg:
        ga_hits_aggregated = ga_hits_aggregated.drop(columns=existing_cols_to_drop_hits_agg, axis = 1)

    # print("-> Предобработка и агрегация данных хитов завершена.", file=sys.stderr)


    # --------------------------------------------------------------------------
    # Объединение датафреймов
    # --------------------------------------------------------------------------
    # print("\n-> Объединение датафреймов...", file=sys.stderr)
    # Не сохраняем промежуточные файлы merged_data.pkl / merged_data_2.pkl в функции предобработки
    # Это опционально и должно быть вне этой функции, если нужно для отладки или хранения
    data = merge_and_save(ga_sessions_filled, ga_hits_aggregated, output_pkl=None) # Убрал сохранение
    del ga_sessions_filled, ga_hits_aggregated; gc.collect()

    if data.empty:
         print("Ошибка: Объединенный датафрейм пуст после слияния.", file=sys.stderr)
         return pd.DataFrame()


    columns_to_drop_from_data = ['device_model','device_browser','device_model_encoded']
    existing_cols_to_drop_data = [col for col in columns_to_drop_from_data if col in data.columns]
    if existing_cols_to_drop_data:
        data = data.drop(columns=existing_cols_to_drop_data, axis = 1)


    # Удаляем действия без пользователей (строки с NaN в 'first_hit_number')
    if 'first_hit_number' in data.columns:
         data.dropna(subset=['first_hit_number'], inplace=True)
    # else:
         # print("'first_hit_number' столбец отсутствует.", file=sys.stderr)

    if data.empty:
        print("Ошибка: Датафрейм пуст после финальной очистки (dropna).", file=sys.stderr)
        return pd.DataFrame()

    # print("\n--- Функция preprocess_data завершена ---", file=sys.stderr)
    return data

# ==============================================================================
# 4. Основной исполняемый код: Замер времени предсказания
# ==============================================================================

if __name__ == "__main__":

    print("Запуск скрипта для замера времени предсказания.", file=sys.stderr)
    print("==============================================================================", file=sys.stderr)

    # Настройка путей к валидационным данным и обученной модели
    file_path_hits_valid = 'ga_hits_valid.pkl' # Файл с валидационными хитами
    file_path_sessions_valid = 'ga_sessions_valid.pkl' # Файл с валидационными сессиями
    model_filepath = 'lgbm_model.pkl' # Файл с обученной моделью

    # Убедимся, что файлы существуют
    if not Path(file_path_hits_valid).exists():
         print(f"Ошибка: Валидационный файл хитов '{file_path_hits_valid}' не найден. Замер невозможен.", file=sys.stderr)
         sys.exit(1)
    if not Path(file_path_sessions_valid).exists():
         print(f"Ошибка: Валидационный файл сессий '{file_path_sessions_valid}' не найден. Замер невозможен.", file=sys.stderr)
         sys.exit(1)
    if not Path(model_filepath).exists():
         print(f"Ошибка: Файл обученной модели '{model_filepath}' не найден. Замер невозможен.", file=sys.stderr)
         sys.exit(1)


    # --- Начало замера времени ---
    start_time = time.time()
    print("\n--- Начало замера времени ---", file=sys.stderr)

    # 1) Прочитать датафреймы
    print("Загрузка валидационных данных...", file=sys.stderr)
    sessions_valid_raw = None
    hits_valid_raw = None
    try:
        sessions_valid_raw = pd.read_pickle(file_path_sessions_valid)
        print(f"Файл '{file_path_sessions_valid}' успешно загружен. Размер: {len(sessions_valid_raw)} строк.", file=sys.stderr)
    except Exception as e:
        print(f"Ошибка при загрузке '{file_path_sessions_valid}': {e}", file=sys.stderr)
        sys.exit(1)

    try:
        hits_valid_raw = pd.read_pickle(file_path_hits_valid)
        print(f"Файл '{file_path_hits_valid}' успешно загружен. Размер: {len(hits_valid_raw)} строк.", file=sys.stderr)
    except Exception as e:
        print(f"Ошибка при загрузке '{file_path_hits_valid}': {e}", file=sys.stderr)
        sys.exit(1)

    if sessions_valid_raw.empty or hits_valid_raw.empty:
         print("Ошибка: Загруженные валидационные датафреймы пусты. Замер невозможен.", file=sys.stderr)
         sys.exit(1)

    # Применяем начальную предобработку 'not set'/'none'
    print("Применение начальной предобработки 'not set'/'none' к валидационным данным...", file=sys.stderr)
    process_notset_none(sessions_valid_raw)
    process_notset_none(hits_valid_raw)


    # 2) Выполнить весь пиплайн предобработки
    print("\nВыполнение полного пайплайна предобработки на валидационных данных...", file=sys.stderr)
    data_valid_preprocessed = preprocess_data(sessions_valid_raw.copy(), hits_valid_raw.copy())

    # Освобождаем память от исходных валидационных данных
    del sessions_valid_raw, hits_valid_raw
    gc.collect()

    if data_valid_preprocessed.empty:
         print("Ошибка: Данные после предобработки пусты. Предсказание невозможно.", file=sys.stderr)
         sys.exit(1)


    # 3) Загрузить обученную модель
    print(f"\nЗагрузка обученной модели из '{model_filepath}'...", file=sys.stderr)
    model_pipeline = None
    try:
        with open(model_filepath, 'rb') as f:
            model_pipeline = pickle.load(f)
        print("Модель успешно загружена.", file=sys.stderr)
    except Exception as e:
        print(f"Ошибка при загрузке модели: {e}", file=sys.stderr)
        sys.exit(1)


    # Подготовка данных для предсказания (удаление целевого столбца, если есть, и session_id)
    # preprocess_data не удаляет целевой столбец и session_id
    print("Подготовка данных для предсказания...", file=sys.stderr)
    X_valid = data_valid_preprocessed.drop(columns=[target_column_name_for_prediction, 'session_id'], errors='ignore')

    if X_valid.empty:
         print("Ошибка: Нет признаков для предсказания после подготовки данных. Замер невозможен.", file=sys.stderr)
         sys.exit(1)

    # 4) Выдача предикта
    print("\nВыполнение предсказания...", file=sys.stderr)
    predictions = None
    try:
        # Используйте .predict() для получения предсказанных классов
        predictions = model_pipeline.predict(X_valid)
        print("Предсказание выполнено.", file=sys.stderr)

        # Если нужны вероятности, используйте .predict_proba()
        # probabilities = model_pipeline.predict_proba(X_valid)
        # print("Предсказанные вероятности получены.")

    except Exception as e:
        print(f"Ошибка при выполнении предсказания: {e}", file=sys.stderr)
        sys.exit(1)

    # --- Конец замера времени ---
    end_time = time.time()
    elapsed_time = end_time - start_time

    print("\n--- Конец замера времени ---", file=sys.stderr)
    print(f"Общее время выполнения пайплайна (загрузка валидации -> предобработка -> загрузка модели -> предсказание): {elapsed_time:.4f} секунд", file=sys.stderr)


    # Вывод первых нескольких предсказаний для проверки
    print("\nПервые 10 предсказанных значений:", file=sys.stderr)
    if predictions is not None:
        print(predictions[:10], file=sys.stderr)
    else:
         print("Предсказания не были получены.", file=sys.stderr)


    print("\n==============================================================================", file=sys.stderr)
    print("Скрипт замера времени предсказания завершен.", file=sys.stderr)
    print("==============================================================================", file=sys.stderr)